In [1]:
import pandas as pd
import torch.backends.cudnn as cudnn
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from base_model import BaseModel
import pickle
import numpy as np
import warnings
import random
import os
warnings.filterwarnings("ignore")
import ast
import torch
import torch.nn.functional as F
from data_utils import get_dataset
from collections import Counter
import torch.nn as nn
from torch.autograd import Variable

In [2]:
seed = 7
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed_all(seed)
device = f"cuda:{0}"
cpu_device = torch.device("cpu:0")
cudnn.benchmark = True

In [3]:
input_dim = 3
dataset_name = "cifar10"
model_name = "resnet18"
num_cls = 10
shadow_num = 1

# Data processing

In [4]:
trainset = get_dataset(dataset_name, train=True)
testset = get_dataset(dataset_name, train=False)    
data_path = f"data/{dataset_name}_data_index_inference.pkl"
    
with open(data_path, 'rb') as f:
    inference_victim_train_list, inference_victim_test_list, inference_attack_split_list = pickle.load(f)

Build Dataset cifar10
Build Dataset cifar10


In [5]:
save_folder_original = f"results/{dataset_name}_{model_name}"
save_folder_compress = f"results_pruning/{dataset_name}_{model_name}"
save_folder_compress_q = f"results_quantization/{dataset_name}_{model_name}"
#save_folder_compress_c = f"results_clustering/{dataset_name}_{model_name}"

In [6]:
victim_train_dataset = Subset(trainset, inference_victim_train_list)
victim_test_dataset = Subset(testset, inference_victim_test_list)
print(f"Victim Train Size: {len(inference_victim_train_list)}, "
        f"Victim Test Size: {len(inference_victim_test_list)}")

Victim Train Size: 10000, Victim Test Size: 10000


In [7]:
victim_train_loader = DataLoader(victim_train_dataset, batch_size=128, shuffle=False)
victim_test_loader = DataLoader(victim_test_dataset, batch_size=128, shuffle=False)

# Model loading

In [8]:
# Load victim model
victim_model_save_folder = save_folder_original + "/victim_model"
victim_model_path = f"{victim_model_save_folder}/best.pth"
victim_model = BaseModel(dataset_name, model_name, num_cls=num_cls, input_dim=input_dim, device=device)
victim_model.load(victim_model_path)

victim_original_in_loss, victim_in_target, victim_original_in_predicts = victim_model.loss_target_predict(victim_train_loader)
victim_original_out_loss, victim_out_target, victim_original_out_predicts = victim_model.loss_target_predict(victim_test_loader)
sort_victim_original_in = torch.argsort(victim_original_in_predicts, dim=1, descending=False)
sort_victim_original_out = torch.argsort(victim_original_out_predicts, dim=1, descending=False)
victim_original_in_predicts = torch.gather(victim_original_in_predicts, 1, sort_victim_original_in)
victim_original_out_predicts = torch.gather(victim_original_out_predicts, 1, sort_victim_original_out)
victim_original_in_loss = victim_original_in_loss.unsqueeze(1)
victim_original_out_loss = victim_original_out_loss.unsqueeze(1)

In [9]:
victim_compress_model_list = []
for i in [0.6, 0.7, 0.8, 0.9]: # different prune levels
    compress_victim_model_save_folder = f"{save_folder_compress}_{i}/victim_model"
    print(f"Load Compress Model from {compress_victim_model_save_folder}")
    victim_compress_model = BaseModel(dataset_name, model_name, num_cls=num_cls, input_dim=input_dim, save_folder=compress_victim_model_save_folder, device=device)
    victim_compress_model.model.load_state_dict(torch.load(f"{compress_victim_model_save_folder}/best.pth"))
    victim_compress_model_list.append(victim_compress_model)

victim_compress_model = BaseModel(dataset_name, model_name, num_cls=num_cls, input_dim=input_dim, device=torch.device("cpu:0"))
victim_compress_model.load_torchscript_model(f"results_quantization/{dataset_name}_{model_name}_int8/victim_model/best.pth")
victim_compress_model_list.append(victim_compress_model)

Load Compress Model from results_pruning/cifar10_resnet18_0.6/victim_model
Load Compress Model from results_pruning/cifar10_resnet18_0.7/victim_model
Load Compress Model from results_pruning/cifar10_resnet18_0.8/victim_model
Load Compress Model from results_pruning/cifar10_resnet18_0.9/victim_model


In [10]:
print(len(victim_compress_model_list))

5


In [11]:
victim_compress_in_predicts_list = []
victim_compress_out_predicts_list = []
victim_compress_in_loss_list = []
victim_compress_out_loss_list = []
victim_compress_in_state_list = []
victim_compress_out_state_list = []
for victim_compress_model in victim_compress_model_list:
    victim_compress_in_loss,victim_in_target, victim_compress_in_predicts = victim_compress_model.loss_target_predict(victim_train_loader)
    victim_compress_out_loss, victim_out_target, victim_compress_out_predicts = victim_compress_model.loss_target_predict(victim_test_loader)
    
    victim_compress_in_predicts =  victim_compress_in_predicts.gather(1, sort_victim_original_in)
    victim_compress_in_predicts = victim_original_in_predicts - victim_compress_in_predicts
    victim_compress_out_predicts =  victim_compress_out_predicts.gather(1, sort_victim_original_out)
    victim_compress_out_predicts = victim_original_out_predicts - victim_compress_out_predicts
    
    victim_compress_in_predicts_list.append(victim_compress_in_predicts)
    victim_compress_out_predicts_list.append(victim_compress_out_predicts)
    victim_compress_in_loss_list.append(victim_compress_in_loss)
    victim_compress_out_loss_list.append(victim_compress_out_loss)

victim_compress_in_loss_list = [x.unsqueeze(1) for x in victim_compress_in_loss_list]  
victim_compress_out_loss_list = [x.unsqueeze(1) for x in victim_compress_out_loss_list]   
victim_compress_in_predicts = torch.cat(victim_compress_in_predicts_list, dim=1)
victim_compress_out_predicts = torch.cat(victim_compress_out_predicts_list, dim=1)
victim_compress_in_loss = torch.cat(victim_compress_in_loss_list, dim=1)
victim_compress_out_loss = torch.cat(victim_compress_out_loss_list, dim=1)

In [12]:
batch_size = 128

In [13]:
shadow_model_list, shadow_train_loader_list, shadow_test_loader_list, shadow_prune_model_group_list = [], [], [], []

for shadow_ind in range(shadow_num):
    attack_train_list, attack_test_list = inference_attack_split_list[shadow_ind]
    shadow_train_dataset = Subset(trainset, attack_train_list)
    shadow_test_dataset = Subset(testset, attack_test_list)
    shadow_train_loader = DataLoader(shadow_train_dataset, batch_size=batch_size, shuffle=False)
    shadow_test_loader = DataLoader(shadow_test_dataset, batch_size=batch_size, shuffle=False)
    shadow_model_path = f"{save_folder_original}/shadow_model_{shadow_ind}/best.pth"
    shadow_model = BaseModel(dataset_name,model_name, num_cls=num_cls, input_dim=input_dim, device=device)
    shadow_model.load(shadow_model_path)
    shadow_model_list.append(shadow_model)
    shadow_train_loader_list.append(shadow_train_loader)
    shadow_test_loader_list.append(shadow_test_loader)
    shadow_prune_model_list = []
    for i in [0.6, 0.7, 0.8, 0.9]:
        compress_shadow_model_save_folder = f"{save_folder_compress}_{i}/shadow_model_{shadow_ind}"
        print(f"Load compress Shadow Model From {compress_shadow_model_save_folder}")
        shadow_compress_model = BaseModel(dataset_name,model_name, num_cls=num_cls, input_dim=input_dim,save_folder=compress_shadow_model_save_folder, device=device)
        shadow_compress_model.model.load_state_dict(torch.load(f"{compress_shadow_model_save_folder}/best.pth"))
        shadow_prune_model_list.append(shadow_compress_model)
    shadow_compress_model = BaseModel(dataset_name, model_name, num_cls=num_cls, input_dim=input_dim, device=torch.device("cpu:0"))
    shadow_compress_model.load_torchscript_model(f"results_quantization/{dataset_name}_{model_name}_int8/shadow_model_{shadow_ind}/best.pth")
    shadow_prune_model_list.append(shadow_compress_model)
    shadow_prune_model_group_list.append(shadow_prune_model_list)

Load compress Shadow Model From results_pruning/cifar10_resnet18_0.6/shadow_model_0
Load compress Shadow Model From results_pruning/cifar10_resnet18_0.7/shadow_model_0
Load compress Shadow Model From results_pruning/cifar10_resnet18_0.8/shadow_model_0
Load compress Shadow Model From results_pruning/cifar10_resnet18_0.9/shadow_model_0


In [14]:
attack_original_in_predicts_list, attack_original_out_predicts_list = [], []
attack_compress_in_predicts_list, attack_compress_out_predicts_list = [], []
attack_original_in_loss_list, attack_original_out_loss_list = [], []
attack_compress_in_loss_list, attack_compress_out_loss_list = [], []
attack_in_targets_list = []
attack_out_targets_list= []

n = 0

for shadow_model, shadow_train_loader, shadow_test_loader in zip(shadow_model_list, shadow_train_loader_list, shadow_test_loader_list):

    attack_original_in_loss,attack_in_target, attack_original_in_predicts = shadow_model.loss_target_predict(shadow_train_loader)
    attack_original_out_loss, attack_out_target, attack_original_out_predicts = shadow_model.loss_target_predict(shadow_test_loader)

    sort_attack_original_in = torch.argsort(attack_original_in_predicts, dim=1, descending=False)
    sort_attack_original_out = torch.argsort(attack_original_out_predicts, dim=1, descending=False)
    attack_original_in_predicts = torch.gather(attack_original_in_predicts, 1, sort_attack_original_in)
    attack_original_out_predicts = torch.gather(attack_original_out_predicts, 1, sort_attack_original_out)
    
    attack_original_in_loss = attack_original_in_loss.unsqueeze(1)
    attack_original_out_loss = attack_original_out_loss.unsqueeze(1)
    
    attack_original_in_loss_list.append(attack_original_in_loss)
    attack_original_out_loss_list.append(attack_original_out_loss)
    
    attack_original_in_predicts_list.append(attack_original_in_predicts)
    attack_original_out_predicts_list.append(attack_original_out_predicts) 
    attack_in_targets_list.append(attack_in_target)
    attack_out_targets_list.append(attack_out_target)  

    each_attack_compress_in_predicts_list = []
    each_attack_compress_out_predicts_list = []
    each_attack_compress_in_loss_list = []
    each_attack_compress_out_loss_list = []

    shadow_prune_models_for_n = shadow_prune_model_group_list[n]
    n = n+1
    for i, shadow_prune_model in enumerate(shadow_prune_models_for_n):
        print(f"Shadow prune model {i} for shadow {n}: {shadow_prune_model}")
        attack_compress_out_loss, attack_out_targets, attack_compress_out_predicts = shadow_compress_model.loss_target_predict(shadow_test_loader)
        attack_compress_in_loss, attack_in_targets, attack_compress_in_predicts = shadow_compress_model.loss_target_predict(shadow_train_loader)

        attack_compress_in_predicts = attack_compress_in_predicts.gather(1, sort_attack_original_in)
        attack_compress_out_predicts =  attack_compress_out_predicts.gather(1, sort_attack_original_out)
        
        attack_compress_in_predicts = attack_original_in_predicts - attack_compress_in_predicts
        attack_compress_out_predicts = attack_original_out_predicts - attack_compress_out_predicts
        
        each_attack_compress_out_predicts_list.append(attack_compress_out_predicts)
        each_attack_compress_in_predicts_list.append(attack_compress_in_predicts)

        each_attack_compress_in_loss_list.append(attack_compress_in_loss)
        each_attack_compress_out_loss_list.append(attack_compress_out_loss)

        
    each_attack_compress_in_loss_list = [x.unsqueeze(1) for x in each_attack_compress_in_loss_list]  # 增加一个维度
    each_attack_compress_out_loss_list = [x.unsqueeze(1) for x in each_attack_compress_out_loss_list]  # 增加一个维度

    each_attack_compress_in_predicts = torch.cat(each_attack_compress_in_predicts_list, dim=1)
    each_attack_compress_out_predicts = torch.cat(each_attack_compress_out_predicts_list, dim=1)
    attack_compress_in_predicts_list.append(each_attack_compress_in_predicts)
    attack_compress_out_predicts_list.append(each_attack_compress_out_predicts)

    each_attack_compress_in_loss = torch.cat(each_attack_compress_in_loss_list, dim=1)
    each_attack_compress_out_loss = torch.cat(each_attack_compress_out_loss_list, dim=1)
    attack_compress_in_loss_list.append(each_attack_compress_in_loss)
    attack_compress_out_loss_list.append(each_attack_compress_out_loss)




attack_original_in_predicts = torch.cat(attack_original_in_predicts_list, dim=0)
attack_original_out_predicts = torch.cat(attack_original_out_predicts_list, dim=0)
attack_compress_in_predicts = torch.cat(attack_compress_in_predicts_list, dim=0)
attack_compress_out_predicts = torch.cat(attack_compress_out_predicts_list, dim=0)
attack_in_targets = torch.cat(attack_in_targets_list, dim=0)
attack_out_targets = torch.cat(attack_out_targets_list, dim=0)

attack_original_in_loss = torch.cat(attack_original_in_loss_list, dim=0)
attack_original_out_loss = torch.cat(attack_original_out_loss_list, dim=0)

attack_compress_in_loss = torch.cat(attack_compress_in_loss_list, dim=0)
attack_compress_out_loss = torch.cat(attack_compress_out_loss_list, dim=0)


Shadow prune model 0 for shadow 1: <base_model.BaseModel object at 0x7fc3e6936a80>
Shadow prune model 1 for shadow 1: <base_model.BaseModel object at 0x7fc29b8910a0>
Shadow prune model 2 for shadow 1: <base_model.BaseModel object at 0x7fc2aea91640>
Shadow prune model 3 for shadow 1: <base_model.BaseModel object at 0x7fc2d68b65a0>
Shadow prune model 4 for shadow 1: <base_model.BaseModel object at 0x7fc2aebc5880>


In [15]:
train_pre = np.concatenate((attack_compress_in_predicts, attack_compress_out_predicts), axis=0)
test_pre = np.concatenate((victim_compress_in_predicts, victim_compress_out_predicts), axis=0)

In [16]:
attack_in_targets = F.one_hot(attack_in_targets, num_classes=num_cls).float() 
attack_out_targets = F.one_hot(attack_out_targets, num_classes=num_cls).float()
train_target = np.concatenate((attack_in_targets.cpu(), attack_out_targets.cpu()), axis=0)

victim_in_targets = F.one_hot(victim_in_target, num_classes=num_cls).float()
victim_out_targets = F.one_hot(victim_out_target, num_classes=num_cls).float()
test_target = np.concatenate((victim_in_targets.cpu(), victim_out_targets.cpu()), axis=0)

In [17]:
train_loss = np.concatenate((attack_compress_in_loss, attack_compress_out_loss), axis=0)
test_loss = np.concatenate((victim_compress_in_loss, victim_compress_out_loss), axis=0)
print(len(train_loss))

20000


In [18]:
num = len(attack_in_targets)
total_num = num*shadow_num 
ones = torch.ones(total_num)
zeros = torch.zeros(total_num)
train_labels = torch.cat((ones, zeros), dim=0)
print(train_labels.shape)

ones = torch.ones(num)
zeros = torch.zeros(num)
test_labels = torch.cat((ones, zeros), dim=0)
print(test_labels.shape)

torch.Size([20000])
torch.Size([20000])


In [19]:
attack_model_name = "RF"
method_name = "CompLeakSR2"

In [20]:
import re

def load_attack_probs(
    save_folder_compress,
    compress_ratio,
    attack_model_name,
    method_name
):
    """
    compress_ratio: float, e.g. 0.6 / 0.7 / 0.8 / 0.9
    """
    csv_path = f"{save_folder_compress}_{compress_ratio}/{attack_model_name}_CompLeakSR.csv"
    results = pd.read_csv(csv_path)

    condition = (
        (results['method'] == method_name) &
        (results['attack_model_name'] == attack_model_name)
    )
    prob = results[condition]

    train_prob = ast.literal_eval(prob['train_prob'].values[0])
    test_prob  = ast.literal_eval(prob['test_prob'].values[0])

    train_prob = np.array(train_prob).reshape(-1, 2)
    test_prob  = np.array(test_prob).reshape(-1, 2)

    return train_prob, test_prob

In [30]:
def build_concat_dataset(
    save_folder_compress,
    ratios,
    attack_model_name,
    method_name
):
    train_list = []
    test_list = []

    for r in ratios:
        if r == 'int8':
            save_folder_compress = save_folder_compress_q
        train_prob, test_prob = load_attack_probs(
            save_folder_compress,
            r,
            attack_model_name,
            method_name
        )
        train_list.append(train_prob)
        test_list.append(test_prob)

    train_prob_combined = np.concatenate(train_list, axis=1)
    test_prob_combined  = np.concatenate(test_list, axis=1)

    train_prob_combined = torch.tensor(train_prob_combined, dtype=torch.float32)
    test_prob_combined  = torch.tensor(test_prob_combined, dtype=torch.float32)

    return train_prob_combined, test_prob_combined

In [31]:
ratios = [0.6, 0.7, 0.8, 0.9, 'int8']

train_prob_combined, test_prob_combined = build_concat_dataset(
    save_folder_compress,
    ratios,
    attack_model_name,
    method_name
)
print(train_prob_combined.shape)
print(len(test_prob_combined))

torch.Size([20000, 10])
20000


In [32]:
class CustomDataset(Dataset):
    def __init__(self, data1, data2, labels):
        self.data1 = data1
        self.data2 = data2
        self.labels = labels
    def __len__(self):
        return len(self.data1)
    def __getitem__(self, idx):
        data1 = self.data1[idx]
        data2 = self.data2[idx]
        label = self.labels[idx]
        
        return data1, data2, label

In [33]:
print(len(train_labels))
print(len(test_loss))

20000
20000


In [34]:
train_dataset = CustomDataset(train_prob_combined, train_loss, train_labels)
test_dataset = CustomDataset(test_prob_combined,test_loss,test_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [39]:
compress_num = 5 # number of compress model

In [40]:
class AttackModel(nn.Module):
    def __init__(self, class_num):
        super(AttackModel, self).__init__()

        self.prob_component = nn.Sequential(
            nn.Linear(2*compress_num, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 64),
        )

        self.loss_component = nn.Sequential(
            nn.Linear(1*compress_num, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 64),
        )

        
        self.encoder_component = nn.Sequential(
           nn.Dropout(p=0.5),
            nn.Linear(64*2 , 256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(64, 2),
        )


    def forward(self, prob, loss):
        prob_component_result = self.prob_component(prob)
        loss_component_result = self.loss_component(loss)
        final_input = torch.cat((prob_component_result, loss_component_result), 1)
        final_result = self.encoder_component(final_input)
        return final_result, final_input

In [41]:
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
def train_mia_attack_model(model, attack_train_loader, optimizer, loss_fn, device):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (input1,input2, member_status) in enumerate(attack_train_loader):
        input1 = input1.to(device)
        input2 = input2.to(device)

        output,_ = model(input1,input2)        
        member_status = member_status.to(device)
        loss = loss_fn(output, member_status.long())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(member_status.view_as(pred)).sum().item()
        
    train_loss /= len(attack_train_loader.dataset)
    accuracy = 100. * correct / len(attack_train_loader.dataset)
    return train_loss, accuracy / 100

def test_mia_attack_model(model, attack_test_loader, loss_fn, device):                 
    model.eval()
    test_loss = 0
    correct = 0
    all_ground_truth = []
    all_pred_probs = []
    final_inputs = []
    
    
    with torch.no_grad():
        for batch_idx, (input1, input2, member_status) in enumerate(attack_test_loader):
            input1 = input1.to(device)
            input2 = input2.to(device)
            #input3 = input3.to(device)
            
            output,final_input = model(input1, input2) 
            final_inputs.append(final_input.detach().cpu())
            member_status = member_status.to(device)

            
            test_loss += loss_fn(output, member_status.long()).item()
            
            probs = torch.softmax(output, dim=1)[:, 1]  
            all_pred_probs.extend(probs.cpu().numpy())  
            all_ground_truth.extend(member_status.cpu().numpy())  
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(member_status.view_as(pred)).sum().item()
    
    test_loss /= len(attack_test_loader.dataset)
    accuracy = 100. * correct / len(attack_test_loader.dataset)
    
    auc = roc_auc_score(all_ground_truth, all_pred_probs)
    

    fpr, tpr, thresholds = roc_curve(all_ground_truth, all_pred_probs)

    fpr_target = 0.001
    interp_func = interp1d(fpr, tpr)
    tpr_at_low_fpr = interp_func(fpr_target)
    final_inputs = torch.cat(final_inputs, dim=0)
    return test_loss, accuracy / 100., auc, tpr_at_low_fpr*100, final_inputs

In [42]:
import torch
import numpy as np
import torch.optim as optim
from sklearn.metrics import roc_auc_score
print(f'-------------------mia------------------')
attack_model = AttackModel(num_cls)
epoch = 100
attack_optimizer = torch.optim.SGD(attack_model.parameters(), 1e-2, momentum=0.9, weight_decay=5e-4)
attack_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(attack_optimizer, T_max=epoch)
attack_model = attack_model.to(device)
loss_fn = nn.CrossEntropyLoss()
best_prec = 0.0
best_auc = 0.0
for epoch in range(epoch):
    train_loss, train_prec = train_mia_attack_model(attack_model, train_loader, attack_optimizer, loss_fn, device)
    val_loss, val_prec, auc, tpr_at_low_fpr, final_inputs = test_mia_attack_model(attack_model, test_loader, loss_fn, device)
    attack_scheduler.step()
    is_best_prec = val_prec > best_prec
    if is_best_prec:
        best_prec = val_prec
        best_auc = auc
    #print(('epoch:{} \t tpr_at_0.001_fpr:{:.4f} \t test_prec:{:.4f} \t test_auc:{:.4f} \t best_prec:{:.4f} \t best_auc:{:.4f}'.format(epoch, tpr_at_low_fpr, val_prec, auc, best_prec, best_auc))

    print('epoch:{} \t tpr_at_0.001_fpr:{:.4f} \t test_prec:{:.4f} \t test_auc:{:.4f} \t best_prec:{:.4f} \t best_auc:{:.4f}'.format(epoch, tpr_at_low_fpr, val_prec, auc, best_prec, best_auc))

-------------------mia------------------
epoch:0 	 tpr_at_0.001_fpr:3.1900 	 test_prec:0.6003 	 test_auc:0.7164 	 best_prec:0.6003 	 best_auc:0.7164
epoch:1 	 tpr_at_0.001_fpr:4.8400 	 test_prec:0.6769 	 test_auc:0.7631 	 best_prec:0.6769 	 best_auc:0.7631
epoch:2 	 tpr_at_0.001_fpr:5.0100 	 test_prec:0.6881 	 test_auc:0.7648 	 best_prec:0.6881 	 best_auc:0.7648
epoch:3 	 tpr_at_0.001_fpr:5.2500 	 test_prec:0.6850 	 test_auc:0.7638 	 best_prec:0.6881 	 best_auc:0.7648
epoch:4 	 tpr_at_0.001_fpr:4.8400 	 test_prec:0.6869 	 test_auc:0.7639 	 best_prec:0.6881 	 best_auc:0.7648
epoch:5 	 tpr_at_0.001_fpr:4.7200 	 test_prec:0.6836 	 test_auc:0.7622 	 best_prec:0.6881 	 best_auc:0.7648
epoch:6 	 tpr_at_0.001_fpr:4.6000 	 test_prec:0.6805 	 test_auc:0.7614 	 best_prec:0.6881 	 best_auc:0.7648
epoch:7 	 tpr_at_0.001_fpr:4.4000 	 test_prec:0.6847 	 test_auc:0.7613 	 best_prec:0.6881 	 best_auc:0.7648
epoch:8 	 tpr_at_0.001_fpr:4.1200 	 test_prec:0.6830 	 test_auc:0.7615 	 best_prec:0.6881 	 bes